In [ ]:
spacy.explain('ADP')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Apple is looking at buying U.K. startup for $1 billion,')
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [67]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Magic Johnson did not play for the Lakers.')
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Magic Johnson 0 13 PERSON
Lakers 35 41 ORG


In [14]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

In [110]:
df = pd.read_json('devset.json',)
subset = df.transpose()
#df.head()
#subset = df[20:40]
subset.head()

,claim,evidence,label
91198,Colin Kaepernick became a starting quarterback...,[],NOT ENOUGH INFO
194462,Tilda Swinton is a vegan.,[],NOT ENOUGH INFO
137334,Fox 2000 Pictures released the film Soul Food.,"[[Soul_Food_-LRB-film-RRB-, 0]]",SUPPORTS
166626,Anne Rice was born in New Jersey.,[],NOT ENOUGH INFO
111897,Telemundo is a English-language television net...,"[[Telemundo, 5], [Telemundo, 4], [Telemundo, 1...",REFUTES


In [89]:
def noun_counts(sentence):
    doc = nlp(sentence)
    count = 0
    for token in doc:
        if token.pos_ == "PROPN" or token.pos_ == "NOUN":
            count +=1
    return count

In [90]:
def numeric_counts(sentence):
    doc = nlp(sentence)
    count = 0
    for token in doc:
        if token.is_alpha == False and token.pos_ != "PUNCT":
            count +=1
    return count

In [91]:
def common_word_counts(sentence):
    doc = nlp(sentence)
    count = 0
    for token in doc:
        if token.is_stop == True:
            count +=1
    return count

In [92]:
def punctuation_counts(sentence):
    doc = nlp(sentence)
    count = 0
    for token in doc:
        if token.lemma_ == ",":
            count +=1
    return count

In [93]:
def person_counts(sentence):
    doc = nlp(sentence)
    count = 0
    for token in doc.ents:
        if token.label_ == "PERSON":
            count +=1
    return count

In [94]:
def organisation_counts(sentence):
    doc = nlp(sentence)
    count = 0
    for token in doc.ents:
        if token.label_ == "ORG":
            count +=1
    return count

In [95]:
def product_counts(sentence):
    doc = nlp(sentence)
    count = 0
    for token in doc.ents:
        if token.label_ == "PRODUCT":
            count +=1
    return count

In [ ]:
%%time
subset['no_of_words'] = subset['claim'].apply(lambda x: len(x.split(' ')))
subset['length_of_claim'] = subset['claim'].apply(lambda x: len(x))
subset['average_word_length'] = subset['claim'].apply(lambda x: (len(x)/len(x.split(' '))))
subset['noun_counts'] = subset['claim'].apply(lambda x: noun_counts(x))
subset['numeric_counts'] = subset['claim'].apply(lambda x: numeric_counts(x))
subset['punctuation_counts'] = subset['claim'].apply(lambda x: punctuation_counts(x))
subset['common_word_counts'] = subset['claim'].apply(lambda x: common_word_counts(x))
subset['person_counts'] = subset['claim'].apply(lambda x: person_counts(x))
subset['organisation_counts'] = subset['claim'].apply(lambda x: organisation_counts(x))
subset['product_counts'] = subset['claim'].apply(lambda x: product_counts(x))
subset.head()

In [87]:
s = subset

In [ ]:
test = pd.read_json('test-unlabelled.json').transpose()
test['no_of_words'] = test['claim'].apply(lambda x: len(x.split(' ')))
test['length_of_claim'] = test['claim'].apply(lambda x: len(x))
test['average_word_length'] = test['claim'].apply(lambda x: (len(x)/len(x.split(' '))))
test['noun_counts'] = test['claim'].apply(lambda x: noun_counts(x))
test['numeric_counts'] = test['claim'].apply(lambda x: numeric_counts(x))
test['punctuation_counts'] = test['claim'].apply(lambda x: punctuation_counts(x))
test['common_word_counts'] = test['claim'].apply(lambda x: common_word_counts(x))
test['person_counts'] = test['claim'].apply(lambda x: person_counts(x))
test['organisation_counts'] = test['claim'].apply(lambda x: organisation_counts(x))
test['product_counts'] = test['claim'].apply(lambda x: product_counts(x))
test.head()

#add evidence to test set

In [ ]:
X_train = s
y_train = test
claim = s['claim']
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(X_train,y_train)
#add evidence using IR library
predictions = model.predict(X_test)
d = {'id': X_test['id'],'claim': claim ,'label': predictions}
output = pd.DataFrame(data=d)
output.to_json('testoutput.json',sep=",",index=False)